In [1]:
!pip install textstat
!pip install sentistrength

In [2]:
import textstat
import pandas as pd
from textblob import TextBlob
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from collections import Counter
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sentistrength import PySentiStr

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/My Drive/preproceesed_data_of_thesis_part1_final_preprocess_v1_first_section_0_to_19999_reviews.csv'
df = pd.read_csv(path)

In [5]:
df

,Unnamed: 0,helpfulness,rate,reviews,response
0,0,236,4,use the app awhile get bunch really good ringt...,0
1,1,138,4,the pretty nice app cool winston beauti crisp ...,0
2,2,87,2,use the app year never ani issue wide variety ...,0
3,3,16,5,love choice beauti pic mani choice kind style ...,0
4,4,236,4,use the app awhile get bunch really good ringt...,0
...,...,...,...,...,...
19829,19994,25,4,sometime get glitch overall the superior platf...,0
19830,19995,24,4,best meet app ever find only problem sometime ...,0
19831,19996,26,4,work sad only see easily group call mobil love...,0
19832,19997,32,1,late app doe work correctly one plu 6t audio o...,1


#add features to dataset (measure different metrics)

In [6]:
def getWordnetPOS(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
# list_of_review_noun=[]
# list_of_review_verb=[]
list_of_review_adj=[]
list_of_review_adv=[]


for column in df[['reviews']]:
  text=df[column].values
  for index in range(len(df)):
    #measure number of noun and adj of a review
    # review_noun_count=0
    # review_verb_count=0
    review_adj_count=0
    review_adv_count=0
    for token in word_tokenize(str(text[index])):
      # if getWordnetPOS(token)=="n":
      #   review_noun_count=review_noun_count+1

      # elif getWordnetPOS(token)=="v":
      #   review_verb_count=review_verb_count+1
      if getWordnetPOS(token)=="a":
        review_adj_count=review_adj_count+1
      elif getWordnetPOS(token)=="r":
        review_adv_count=review_adv_count+1
    # list_of_review_noun.append(review_noun_count)
    # list_of_review_verb.append(review_verb_count)
    list_of_review_adj.append(review_adj_count)
    list_of_review_adv.append(review_adv_count)
# df['review_noun_count']=list_of_review_noun
# df['review_verb_count']=list_of_review_verb
df['review_adj_count']=list_of_review_adj 
df['review_adv_count']=list_of_review_adv

# measure sum of tf-idf value for each review (measure review informativeness)

In [7]:
corpus=df['reviews'].tolist()
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(corpus)
df1 = pd.DataFrame(x.toarray(), columns=tfidf.get_feature_names())
df1["sum_of_tfidf"] = df1.sum(axis=1)

In [8]:
result = pd.concat([df,df1[["sum_of_tfidf"]]], axis=1, sort=False)
result

,Unnamed: 0,helpfulness,rate,reviews,response,review_adj_count,review_adv_count,sum_of_tfidf
0,0,236,4,use the app awhile get bunch really good ringt...,0,3,3,4.221928
1,1,138,4,the pretty nice app cool winston beauti crisp ...,0,7,6,4.620335
2,2,87,2,use the app year never ani issue wide variety ...,0,10,7,6.198820
3,3,16,5,love choice beauti pic mani choice kind style ...,0,10,1,5.531627
4,4,236,4,use the app awhile get bunch really good ringt...,0,3,3,4.221928
...,...,...,...,...,...,...,...,...
19829,19994,25,4,sometime get glitch overall the superior platf...,0,11,3,4.736782
19830,19995,24,4,best meet app ever find only problem sometime ...,0,5,17,4.461454
19831,19996,26,4,work sad only see easily group call mobil love...,0,8,7,6.212699
19832,19997,32,1,late app doe work correctly one plu 6t audio o...,1,3,8,4.858913


In [9]:
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [10]:
result.to_csv('/drive/My Drive/preproceesed_data_of_thesis_part1_first-section-reviews with sum of tf-idf and adj and adv.csv')